In [1]:
import pandas as pd 

In [2]:
planillas_hist=pd.read_csv('0.0.planillas.hist.COMPLETE',index_col=0)

In [3]:
planillas_hist

,Dominio,Planilla,Taller,Tipo Uso,Fecha,Resultado,Versión RTO
0,AA000HG,1173505,4,PX,14/11/2022,Apto,2014
1,AA000HG,1124182,4,PX,05/11/2020,Apto,2014
2,AA000HG,1053004,65,PX,04/09/2018,Apto,2014
3,AA001GK,1154432,4,PX,13/01/2022,Apto,2014
4,AA001GK,1066213,65,PX,07/01/2020,Apto,2014
...,...,...,...,...,...,...,...
11690,XOO415,1037818,4,PX,19/08/2016,Apto,2014
11691,XOO981,1074450,4,PX,17/04/2018,Apto,2014
11692,XOS697,1039212,4,PX,13/09/2016,Apto,2014
11693,XOS702,1052067,4,PX,01/04/2017,Condicional,2014


# Importar librerias

In [4]:
import time 

In [5]:
import requests

In [6]:
from tqdm.notebook import tqdm

In [7]:
from bs4 import BeautifulSoup

# Definir funciones

Request detalles de revision

In [8]:
def get_detalles(nro,codTaller,versionRTO):#input is in string format 
    global soup
    url = "https://rto.cent.gov.ar/rto/RTO/verDetallePlanillaRevision"

    payload = "nroPlanilla="+nro+"&codTaller="+codTaller+"&versionRTO="+versionRTO
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:108.0) Gecko/20100101 Firefox/108.0",
        "Accept": "text/javascript, text/html, application/xml, text/xml, */*",
        "Accept-Language": "es-AR,es;q=0.8,en-US;q=0.5,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br",
        "X-Requested-With": "XMLHttpRequest",
        "X-Prototype-Version": "1.6.1",
        "Content-type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://rto.cent.gov.ar",
        "Connection": "keep-alive",
        "Referer": "https://rto.cent.gov.ar/rto/RTO/planillaDiaria",
        "Cookie": cookie,
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin"
    }

    response = requests.request("POST", url, data=payload, headers=headers)
    #print response text (test)
    #print(response.text)
    soup=BeautifulSoup(response.text,'lxml')

Limpiar soup para detalles 

In [9]:
def clean_data(tag):# returns a list of values: tags
        tags_datos=soup.select(tag) #data is inside the tag td
        global datos
        datos=[] 
        for data in tags_datos:
            datos.append(data.get_text(strip=True)) #make a list with the values 
        return(datos)

Limpiar soup para anomalias 

In [10]:
def clean_data_anomalias(tag_anomalias):# returns a list of values: tags
        tags_datos_anomalias=soup.select(tag_anomalias) #data is inside the tag td
        global datos_anomalias
        datos_anomalias=[] 
        for data in tags_datos_anomalias:
            datos_anomalias.append(data.get_text(strip=True)) #make a list with the values 
        return(datos_anomalias)

Crear lista de diccionarios con los detalles: cada diccionario es una fila 

In [11]:
def create_rows_detalles():
    global df
    global rows
    #make dictionary with only the relevant elements 
    #Search Label in the list,if it doesnt exist, returns empty string 
    if 'Número:'in datos:
        Planilla=datos[datos.index('Número:')+1]
    else:
        Planilla=''
    if 'Dominio:'in datos:
        Dominio=datos[datos.index('Dominio:')+1]
    else:
        Dominio=''
    if 'Año Modelo:'in datos:
        Modelo=datos[datos.index('Año Modelo:')+1]
    else:
        Modelo=''
    if 'Convenio:'in datos:
        Convenio=datos[datos.index('Convenio:')+1]
    else:
        Convenio=''
    if 'Resultado:'in datos:
        Resultado=datos[datos.index('Resultado:')+1]
    else:
        Resultado=''


    rows.append({'Planilla':Planilla,
     'Dominio:':Dominio,
     'Modelo':Modelo,
     'Convenio:':Convenio,
     'Resultado':Resultado
    })#make dict 

Crear lista de anomalias:Para cada anomalia es una fila

In [12]:
def create_rows_anomalias(nro_planilla):
    global df_anomalias
    global rows_anomalias
    if len(datos_anomalias)>0:
        a=int(0)
        for i in range (0,int((len(datos_anomalias))/4)):
            rows_anomalias.append([nro_planilla,datos_anomalias[a],datos_anomalias[a+1],datos_anomalias[a+2],datos_anomalias[a+3]])
            a+=int(4)
    else:
        rows_anomalias.append([nro_planilla,None,None,None,None])

Crear dataframe (df) de detalles 

In [13]:
def create_df_detalles():# returns a dataframe df 
    global df 
    df=pd.DataFrame(rows)
    return(df)

Crear dataframe (df_anomalias) de anomalias

In [14]:
def create_df_anomalias():# returns a dataframe df 
    global columns_anomalias
    global df_anomalias 
    columns_anomalias=['Planilla','Código','Gravedad','Descripción','Observaciones']
    df_anomalias=pd.DataFrame()
    df_anomalias=pd.DataFrame(rows_anomalias,columns=columns_anomalias)
    return(df_anomalias)

Utilizando las funciones anteriores. las unimos en una unica funcion. Iteramos en la lista de Nroplanillas y nroTaller

#  FUNCION SCRAPE

In [15]:
def scrape(lista_planillas):
    global rows
    global rows_anomalias
    rows=[]
    rows_anomalias=[] 
    tag='label'#tag detalles planilla  
    tag_anomalias='td'#tag anomalias
    for i in tqdm(lista_planillas):
        get_detalles(str(i[0]),str(i[1]),str(i[2]))
        clean_data(tag)
        create_rows_detalles()
        clean_data_anomalias(tag_anomalias)
        create_rows_anomalias(i[0])
    create_df_detalles()# returns dataframe df
    create_df_anomalias()#returns dataframe df_anomalias de anomalias 
    return([df['Planilla'].nunique(),df_anomalias['Planilla'].nunique()])

In [16]:
def scrape_test(lista_planillas):
    global rows
    global rows_anomalias
    rows=[]
    rows_anomalias=[] 
    tag='label'#tag detalles planilla  
    tag_anomalias='td'#tag anomalias
    for nro_planilla in tqdm(lista_planillas):
        get_detalles(nro_planilla[0],str(nro_planilla[1]))
        clean_data(tag)
        create_rows_detalles()
        clean_data_anomalias(tag_anomalias)
        create_rows_anomalias(nro_planilla[0])
    create_df_detalles()# returns dataframe df
    create_df_anomalias()#returns dataframe df_anomalias de anomalias 
    return([df['Planilla'].nunique(),df_anomalias['Planilla'].nunique()])

Guardar los dataframes en archivos .csv

In [17]:
def to_csv(num):
    filename=str(num)+'planillas.detalle'
    df.to_csv(filename)
    filename2=str(num)+'Anomalias'
    df_anomalias.to_csv(filename2)

Dividimos la lista de planillas en N listas. En caso de error no perdemos todo 

In [18]:
def div_lista(lista,part):
    global lista_part
    n=int(len(lista)/part)
    lista_part=[lista[i:i+n]for i in range(0,len(lista),n)]
    return(lista_part)
    
    

Definimos una nueva funcion Scrape que crea un csv por cada particion de la lista de planillas. Esto nos ayuda a no tener que comenzar de 0 si el server se cae o se corta internet 

In [19]:
def scrape_part(lista,part,start):
    div_lista(lista,part)
    for i in tqdm(range(0+start,part)):
        scrape(lista_part[i])
        to_csv(i)


Definimos una funcion para unir los archivos csv en un unico Dataframe y guardar como csv

In [20]:
def join_csv(part):
    global df_det
    global detalles
    global df_anom
    global anomalias
    detalles=pd.DataFrame()
    anomalias=pd.DataFrame()
    for i in range(0,part):
        df_det=pd.read_csv(str(i)+'planillas.detalle',index_col=0)
        detalles=pd.concat([detalles,df_det])
        df_anom=pd.read_csv(str(i)+'Anomalias',index_col=0)
        anomalias=pd.concat([anomalias,df_anom])        
    detalles.to_csv('0.0.planillas.detalle.COMPLETE')
    anomalias.to_csv('0.0.Anomalias.COMPLETE')
    


# Actualizar cookie

In [21]:
cookie="JSESSIONID=6B255F63B411B029C38BC4B028E1ECC4"

# Lista de planillas para test

In [22]:
planillas_lista=planillas_hist.loc[:,['Planilla','Taller','Versión RTO']]

In [23]:
planillas_lista=planillas_lista.values.tolist()

In [24]:
planillas_lista=planillas_lista[:5]
planillas_lista

[[1173505, 4, 2014],
 [1124182, 4, 2014],
 [1053004, 65, 2014],
 [1154432, 4, 2014],
 [1066213, 65, 2014]]

In [25]:
div_lista(planillas_lista,2)

[[[1173505, 4, 2014], [1124182, 4, 2014]],
 [[1053004, 65, 2014], [1154432, 4, 2014]],
 [[1066213, 65, 2014]]]

In [26]:
scrape_part(planillas_lista,2,0)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
df

,Planilla,Dominio:,Modelo,Convenio:,Resultado
0,1053004,AA000HG,2016,UP Nacional(Nacional-Argentina),Apto
1,1154432,AA001GK,2016,UP Chaco(Provincial-Chaco),Apto


In [28]:
df_anomalias

,Planilla,Código,Gravedad,Descripción,Observaciones
0,1053004,09010302,L,Carrocería > Exterior > Chapa Patente > Faltan...,ESTADO
1,1154432,09010302,L,Carrocería > Exterior > Chapa Patente > Patent...,CHAPA DELANTERA


In [29]:
join_csv(2)

In [30]:
detalles

,Planilla,Dominio:,Modelo,Convenio:,Resultado
0,1173505,AA000HG,2016,UP Chaco(Provincial-Chaco),Apto
1,1124182,AA000HG,2016,UP Chaco(Provincial-Chaco),Apto
0,1053004,AA000HG,2016,UP Nacional(Nacional-Argentina),Apto
1,1154432,AA001GK,2016,UP Chaco(Provincial-Chaco),Apto


In [31]:
anomalias

,Planilla,Código,Gravedad,Descripción,Observaciones
0,1173505,20502,L,"Tren Delantero, Tren Trasero, Suspensión > Amo...",AMORTIGUADORES TRASEROS
1,1173505,20506,L,"Tren Delantero, Tren Trasero, Suspensión > Amo...",AMORTIGUADORES TRASEROS
2,1173505,9010302,L,Carrocería > Exterior > Chapa Patente > Patent...,Patente en mal estado TRASERA
3,1124182,9010302,L,Carrocería > Exterior > Chapa Patente > Patent...,TRASERA
0,1053004,9010302,L,Carrocería > Exterior > Chapa Patente > Faltan...,ESTADO
1,1154432,9010302,L,Carrocería > Exterior > Chapa Patente > Patent...,CHAPA DELANTERA


# Scrape completo

In [32]:
planillas_lista=planillas_hist.loc[:,['Planilla','Taller','Versión RTO']] #numero de planillas: 265738
planillas_lista.drop_duplicates(inplace=True)#numero de planillas 265724
planillas_lista.value_counts()

Planilla  Taller  Versión RTO
14        249     2003           1
1137639   123     2014           1
1137618   124     2014           1
1137619   4       2014           1
1137620   4       2014           1
                                ..
1065672   50      2014           1
1065673   4       2014           1
1065674   4       2014           1
1065675   4       2014           1
1322406   129     2014           1
Length: 265724, dtype: int64

In [33]:
planillas_lista=planillas_lista.values.tolist()

In [34]:
planillas_lista

[[1173505, 4, 2014],
 [1124182, 4, 2014],
 [1053004, 65, 2014],
 [1154432, 4, 2014],
 [1066213, 65, 2014],
 [1256651, 123, 2014],
 [1125455, 4, 2014],
 [1138694, 123, 2014],
 [1177220, 4, 2014],
 [1177004, 4, 2014],
 [1164902, 4, 2014],
 [1153142, 4, 2014],
 [1138906, 4, 2014],
 [1225542, 123, 2014],
 [1096922, 4, 2014],
 [1093923, 84, 2014],
 [1019789, 126, 2014],
 [1052328, 4, 2014],
 [1094979, 4, 2014],
 [1104184, 4, 2014],
 [1003649, 240, 2014],
 [1098176, 4, 2014],
 [1137228, 4, 2014],
 [1096055, 4, 2014],
 [1138126, 4, 2014],
 [1097154, 4, 2014],
 [1136039, 4, 2014],
 [1096410, 4, 2014],
 [1273629, 123, 2014],
 [1101128, 4, 2014],
 [1139275, 4, 2014],
 [1098437, 4, 2014],
 [1139792, 4, 2014],
 [1096229, 4, 2014],
 [1137226, 4, 2014],
 [1096689, 4, 2014],
 [1136325, 4, 2014],
 [1095064, 4, 2014],
 [1080320, 65, 2014],
 [1099951, 4, 2014],
 [1143003, 4, 2014],
 [1095105, 4, 2014],
 [1171662, 4, 2014],
 [1098948, 4, 2014],
 [1137093, 4, 2014],
 [1155018, 123, 2014],
 [1167475, 4, 20

In [35]:
len(planillas_lista)

265724

In [36]:
scrape_part(planillas_lista,100,96)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2657 [00:00<?, ?it/s]

  0%|          | 0/2657 [00:00<?, ?it/s]

  0%|          | 0/2657 [00:00<?, ?it/s]

  0%|          | 0/2657 [00:00<?, ?it/s]

In [41]:
join_csv(99)

# Verificar Datos Detalles planillas

In [38]:
df_detalles_parcial=pd.read_csv(str(100)+'planillas.detalle',index_col=0)
df_detalles_parcial

FileNotFoundError: [Errno 2] No such file or directory: '100planillas.detalle'

# Verificar datos anomalias

In [39]:
df_anomalias_parcial=pd.read_csv(str(99)+'Anomalias',index_col=0)
df_anomalias_parcial

,Planilla,Código,Gravedad,Descripción,Observaciones
0,1074546,010303,L,Sistema de Dirección > Caja de Dirección > Pér...,MOTOR
1,1074546,07020302,L,Sistema Eléctrico > Sistema de Iluminación y S...,DISPOSITIVO
2,1074546,09010302,L,Carrocería > Exterior > Chapa Patente > Faltan...,DELANTERA
3,1074546,090401,L,"Carrocería > Parabrisas. Inexistente, Visibili...",PARABRISAS
4,1074467,010401,M,"Sistema de Dirección > Barras, Brazos, Amortig...",EXTREMO BARRA LARGA
...,...,...,...,...,...
7129,1093751,05020207,L,Emisión de Contaminantes > Humo y Gases de Esc...,NaN
7130,1090031,010303,M,Sistema de Dirección > Caja de Dirección > Pér...,MOTOR
7131,1090031,020303,M,"Tren Delantero, Trasero, Suspensión > Manoplas...",BUJE TREN TRASERO
7132,1090031,020505,M,"Tren Delantero, Trasero, Suspensión > Amortigu...",AMOR DELANTERO IZQ


# Leer csv completo y analizar

detalles

In [42]:
detalles=pd.read_csv('0.0.planillas.detalle.COMPLETE',index_col=0)
detalles

,Planilla,Dominio:,Modelo,Convenio:,Resultado
0,1173505.0,AA000HG,2016.0,UP Chaco(Provincial-Chaco),Apto
1,1124182.0,AA000HG,2016.0,UP Chaco(Provincial-Chaco),Apto
0,1053004.0,AA000HG,2016.0,UP Nacional(Nacional-Argentina),Apto
1,1154432.0,AA001GK,2016.0,UP Chaco(Provincial-Chaco),Apto
0,1120111.0,AA468SX,2016.0,Mercosur G (Mercosur-Mercosur),Apto
...,...,...,...,...,...
2652,NaN,NaN,NaN,NaN,NaN
2653,NaN,NaN,NaN,NaN,NaN
2654,NaN,NaN,NaN,NaN,NaN
2655,1096560.0,WKG856,1989.0,Mercosur G (Mercosur-Mercosur),Apto


In [43]:
detalles.nunique()

Planilla     181075
Dominio:      73263
Modelo           66
Convenio:        31
Resultado         3
dtype: int64

anomalias

In [44]:
anomalias=pd.read_csv('0.0.Anomalias.COMPLETE',index_col=0)
anomalias

,Planilla,Código,Gravedad,Descripción,Observaciones
0,1173505,20502,L,"Tren Delantero, Tren Trasero, Suspensión > Amo...",AMORTIGUADORES TRASEROS
1,1173505,20506,L,"Tren Delantero, Tren Trasero, Suspensión > Amo...",AMORTIGUADORES TRASEROS
2,1173505,9010302,L,Carrocería > Exterior > Chapa Patente > Patent...,Patente en mal estado TRASERA
3,1124182,9010302,L,Carrocería > Exterior > Chapa Patente > Patent...,TRASERA
0,1053004,9010302,L,Carrocería > Exterior > Chapa Patente > Faltan...,ESTADO
...,...,...,...,...,...
6927,1095335,03030401,G,Frenos > Pruebas del Sistema de Freno > Presta...,FRENO DE MANO
6928,1095335,030501,M,Frenos > Freno de Estacionamiento > Recorrido ...,NaN
6929,1095335,040102,G,"Chasis > Bastidor del Chasis > Torceduras, Dob...",CHASIS CON SOLDADURAS
6930,1095335,09010302,L,Carrocería > Exterior > Chapa Patente > Faltan...,NaN


In [45]:
anomalias.nunique()

Planilla         199808
Código              769
Gravedad            151
Descripción         737
Observaciones     46455
dtype: int64